In [58]:
import os

import pandas as pd
import numpy as np
import datetime as dt

import xgboost
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, QuantileTransformer, StandardScaler

# 1. Load data

## 1.1 Load data_forecast

In [2]:
path_data_forecast='data_forecast.csv'
data_forecast=pd.read_csv(path_data_forecast)

## 1.2 Load forecast_kurs_result

In [3]:
path_forecast_kurs='forecast_kurs_result.csv'
forecast_kurs_result=pd.read_csv(path_forecast_kurs)

## 1.3 Load google_trends_result_forecasting

In [4]:
path_google_trends_forecast='google_trends_result_forecasting.csv'
google_trends_forecast=pd.read_csv(path_google_trends_forecast)

# 2. Preprocessing data

## 2.1 Checking data

### 2.1.1 data_forecast

In [5]:
data_forecast.shape

(40664, 8)

In [6]:
data_forecast.isnull().sum()

Tanggal          0
Bahan_pangan     0
Daerah           0
Harga_lag_1      0
Harga_lag_7      0
Harga_lag_30     0
Google_trends    0
Harga            0
dtype: int64

In [7]:
data_forecast.duplicated().sum()

np.int64(0)

### 2.1.2 forecast_kurs_result

In [8]:
forecast_kurs_result.shape

(40664, 8)

In [9]:
forecast_kurs_result.isnull().sum()

Year       0
Month      0
Day        0
Weekday    0
MYRUSD     0
SGDUSD     0
THBUSD     0
USDIDR     0
dtype: int64

In [10]:
forecast_kurs_result.duplicated().sum()

np.int64(40572)

In [11]:
forecast_kurs_result=forecast_kurs_result.drop_duplicates(ignore_index=True)

In [12]:
forecast_kurs_result.duplicated().sum()

np.int64(0)

In [13]:
forecast_kurs_result.shape

(92, 8)

### 2.1.3 google_trends_result_forecasting

In [14]:
google_trends_forecast.shape

(40664, 7)

In [15]:
google_trends_forecast.isnull().sum()

Bahan_pangan     0
Daerah           0
Year             0
Month            0
Day              0
Weekday          0
Google_trends    0
dtype: int64

In [16]:
google_trends_forecast.duplicated().sum()

np.int64(0)

## 2.2 Merge Data

In [17]:
data_forecast.head()

,Tanggal,Bahan_pangan,Daerah,Harga_lag_1,Harga_lag_7,Harga_lag_30,Google_trends,Harga
0,2024-10-01,Bawang Merah,Aceh,28960,27700,26390,0,0
1,2024-10-01,Bawang Merah,Bali,24230,24240,21250,0,0
2,2024-10-01,Bawang Merah,Banten,27940,25910,25520,0,0
3,2024-10-01,Bawang Merah,Bengkulu,27390,26960,23840,0,0
4,2024-10-01,Bawang Merah,DI Yogyakarta,24450,22240,19600,0,0


In [18]:
google_trends_forecast.head()

,Bahan_pangan,Daerah,Year,Month,Day,Weekday,Google_trends
0,0,0,2024,10,1,1,35.722317
1,0,1,2024,10,1,1,40.506355
2,0,2,2024,10,1,1,30.250156
3,0,3,2024,10,1,1,34.963135
4,0,4,2024,10,1,1,34.696740


In [19]:
forecast_kurs_result.head()

,Year,Month,Day,Weekday,MYRUSD,SGDUSD,THBUSD,USDIDR
0,2024,10,1,1,3379.1950,3379.1950,430.08084,14725.526
1,2024,10,2,2,3380.1558,3380.1558,434.03998,14800.794
2,2024,10,3,3,3380.1357,3380.1357,433.72302,14806.403
3,2024,10,4,4,3405.0810,3405.0810,430.43744,14379.611
4,2024,10,5,5,3423.1080,3423.1080,432.58188,14379.490


In [20]:
data_forecast['Google_trends'] = google_trends_forecast.Google_trends

In [21]:
data_forecast['Tanggal']=pd.to_datetime(data_forecast['Tanggal'], errors='coerce')

In [22]:
data_forecast.head()

,Tanggal,Bahan_pangan,Daerah,Harga_lag_1,Harga_lag_7,Harga_lag_30,Google_trends,Harga
0,2024-10-01,Bawang Merah,Aceh,28960,27700,26390,35.722317,0
1,2024-10-01,Bawang Merah,Bali,24230,24240,21250,40.506355,0
2,2024-10-01,Bawang Merah,Banten,27940,25910,25520,30.250156,0
3,2024-10-01,Bawang Merah,Bengkulu,27390,26960,23840,34.963135,0
4,2024-10-01,Bawang Merah,DI Yogyakarta,24450,22240,19600,34.696740,0


In [23]:
data_forecast['Google_trends'] = data_forecast['Google_trends'].apply(lambda x: round(x))

In [24]:
data_forecast.head()

,Tanggal,Bahan_pangan,Daerah,Harga_lag_1,Harga_lag_7,Harga_lag_30,Google_trends,Harga
0,2024-10-01,Bawang Merah,Aceh,28960,27700,26390,36,0
1,2024-10-01,Bawang Merah,Bali,24230,24240,21250,41,0
2,2024-10-01,Bawang Merah,Banten,27940,25910,25520,30,0
3,2024-10-01,Bawang Merah,Bengkulu,27390,26960,23840,35,0
4,2024-10-01,Bawang Merah,DI Yogyakarta,24450,22240,19600,35,0


In [25]:
date_start=dt.datetime(year=2024, month=10, day=1)
date_end=dt.datetime(year=2024, month=12, day=31)

In [27]:
for date in [date_start + dt.timedelta(days=x) for x in range((date_end - date_start).days + 1)]:
    rate_myr = forecast_kurs_result.loc[(forecast_kurs_result['Month']==date.month) & (forecast_kurs_result['Day']==date.day), 'MYRUSD'].values
    rate_thb = forecast_kurs_result.loc[(forecast_kurs_result['Month']==date.month) & (forecast_kurs_result['Day']==date.day), 'THBUSD'].values
    rate_sgd = forecast_kurs_result.loc[(forecast_kurs_result['Month']==date.month) & (forecast_kurs_result['Day']==date.day), 'SGDUSD'].values
    rate_usd = forecast_kurs_result.loc[(forecast_kurs_result['Month']==date.month) & (forecast_kurs_result['Day']==date.day), 'USDIDR'].values
    if len(rate_myr) > 0:
        data_forecast.loc[data_forecast['Tanggal'] == date, 'MYRUSD'] = round(rate_myr[0])
    if len(rate_thb) > 0:
        data_forecast.loc[data_forecast['Tanggal'] == date, 'THBUSD'] = round(rate_thb[0])
    if len(rate_usd) > 0:
        data_forecast.loc[data_forecast['Tanggal'] == date, 'USDIDR'] = round(rate_usd[0])
    if len(rate_sgd) > 0:
        data_forecast.loc[data_forecast['Tanggal'] == date, 'SGDIDR'] = round(rate_sgd[0])

In [28]:
data_forecast.head()

,Tanggal,Bahan_pangan,Daerah,Harga_lag_1,Harga_lag_7,Harga_lag_30,Google_trends,Harga,MYRUSD,THBUSD,USDIDR,SGDIDR
0,2024-10-01,Bawang Merah,Aceh,28960,27700,26390,36,0,3379.0,430.0,14726.0,3379.0
1,2024-10-01,Bawang Merah,Bali,24230,24240,21250,41,0,3379.0,430.0,14726.0,3379.0
2,2024-10-01,Bawang Merah,Banten,27940,25910,25520,30,0,3379.0,430.0,14726.0,3379.0
3,2024-10-01,Bawang Merah,Bengkulu,27390,26960,23840,35,0,3379.0,430.0,14726.0,3379.0
4,2024-10-01,Bawang Merah,DI Yogyakarta,24450,22240,19600,35,0,3379.0,430.0,14726.0,3379.0


In [29]:
data_forecast.tail()

,Tanggal,Bahan_pangan,Daerah,Harga_lag_1,Harga_lag_7,Harga_lag_30,Google_trends,Harga,MYRUSD,THBUSD,USDIDR,SGDIDR
40659,2024-12-31,Tepung Terigu (Curah),Sulawesi Tenggara,0,0,0,38,0,3508.0,445.0,15744.0,3508.0
40660,2024-12-31,Tepung Terigu (Curah),Sulawesi Utara,0,0,0,38,0,3508.0,445.0,15744.0,3508.0
40661,2024-12-31,Tepung Terigu (Curah),Sumatera Barat,0,0,0,37,0,3508.0,445.0,15744.0,3508.0
40662,2024-12-31,Tepung Terigu (Curah),Sumatera Selatan,0,0,0,37,0,3508.0,445.0,15744.0,3508.0
40663,2024-12-31,Tepung Terigu (Curah),Sumatera Utara,0,0,0,37,0,3508.0,445.0,15744.0,3508.0


## 2.3 Save data

In [70]:
output_file='data_forecast_merged.csv'
if not os.path.exists(output_file):
    data_forecast.to_csv(output_file, index=False)

In [120]:
data_forecast = pd.read_csv('data_forecast_merged.csv')

## 2.4 Data Exploration

In [121]:
data_forecast.describe()

,Harga_lag_1,Harga_lag_7,Harga_lag_30,Google_trends,Harga,MYRUSD,THBUSD,USDIDR,SGDIDR
count,40664.000000,40664.000000,40664.000000,40664.000000,40664.0,40664.000000,40664.000000,40664.000000,40664.000000
mean,377.090055,2645.980720,11766.499361,37.511583,0.0,3377.836957,428.206522,14871.086957,3377.836957
std,4939.236626,12879.063617,25160.136301,8.328190,0.0,58.889026,7.267554,565.728632,58.889026
min,0.000000,0.000000,0.000000,-2.000000,0.0,3294.000000,409.000000,14266.000000,3294.000000
25%,0.000000,0.000000,0.000000,37.000000,0.0,3324.000000,423.000000,14399.500000,3324.000000
50%,0.000000,0.000000,0.000000,38.000000,0.0,3377.500000,428.500000,14612.000000,3377.500000
75%,0.000000,0.000000,16330.000000,39.000000,0.0,3427.000000,434.000000,15597.250000,3427.000000
max,158210.000000,162480.000000,162480.000000,81.000000,0.0,3514.000000,445.000000,15773.000000,3514.000000


In [122]:
data_forecast.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40664 entries, 0 to 40663
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Tanggal        40664 non-null  object 
 1   Bahan_pangan   40664 non-null  object 
 2   Daerah         40664 non-null  object 
 3   Harga_lag_1    40664 non-null  int64  
 4   Harga_lag_7    40664 non-null  int64  
 5   Harga_lag_30   40664 non-null  int64  
 6   Google_trends  40664 non-null  int64  
 7   Harga          40664 non-null  int64  
 8   MYRUSD         40664 non-null  float64
 9   THBUSD         40664 non-null  float64
 10  USDIDR         40664 non-null  float64
 11  SGDIDR         40664 non-null  float64
dtypes: float64(4), int64(5), object(3)
memory usage: 3.7+ MB


In [123]:
data_forecast['Tanggal'] = pd.to_datetime(data_forecast['Tanggal'])

In [124]:
bhn_pgn_label_encoder = LabelEncoder()
daerah_label_encoder = LabelEncoder()

data_forecast['Bahan_pangan'] = bhn_pgn_label_encoder.fit_transform(data_forecast['Bahan_pangan'])
data_forecast['Daerah'] = daerah_label_encoder.fit_transform(data_forecast['Daerah'])

In [125]:
# Buat DataFrame untuk mapping Bahan Pangan
mapping_bp = pd.DataFrame({
    "Label": range(len(bhn_pgn_label_encoder.classes_)),
    "Bahan Pangan": bhn_pgn_label_encoder.classes_
})

# Buat DataFrame untuk mapping Daerah
mapping_daerah = pd.DataFrame({
    "Label": range(len(daerah_label_encoder.classes_)),
    "Daerah": daerah_label_encoder.classes_
})

# Tampilkan dalam bentuk tabel
print("Mapping Bahan Pangan:")
print(mapping_bp.to_string(index=False))  # Hapus index agar lebih rapi

print("\nMapping Daerah:")
print(mapping_daerah.to_string(index=False))

Mapping Bahan Pangan:
 Label                    Bahan Pangan
     0                    Bawang Merah
     1            Bawang Putih Bonggol
     2                    Beras Medium
     3                   Beras Premium
     4            Cabai Merah Keriting
     5               Cabai Rawit Merah
     6                 Daging Ayam Ras
     7               Daging Sapi Murni
     8                   Gula Konsumsi
     9             Minyak Goreng Curah
    10 Minyak Goreng Kemasan Sederhana
    11                  Telur Ayam Ras
    12           Tepung Terigu (Curah)

Mapping Daerah:
 Label                    Daerah
     0                      Aceh
     1                      Bali
     2                    Banten
     3                  Bengkulu
     4             DI Yogyakarta
     5               DKI Jakarta
     6                 Gorontalo
     7                     Jambi
     8                Jawa Barat
     9               Jawa Tengah
    10                Jawa Timur
    11          Kal

In [126]:
data_forecast['Year'] = data_forecast['Tanggal'].dt.year
data_forecast['Month'] = data_forecast['Tanggal'].dt.month
data_forecast['Day'] = data_forecast['Tanggal'].dt.day
data_forecast['Weekday'] = data_forecast['Tanggal'].dt.weekday  # Senin (0) - Minggu (6)

In [127]:
data_forecast = data_forecast.drop(columns=['Tanggal'])

In [128]:
data_forecast.head()

,Bahan_pangan,Daerah,Harga_lag_1,Harga_lag_7,Harga_lag_30,...,SGDIDR,Year,Month,Day,Weekday
0,0,0,28960,27700,26390,...,3379.0,2024,10,1,1
1,0,1,24230,24240,21250,...,3379.0,2024,10,1,1
2,0,2,27940,25910,25520,...,3379.0,2024,10,1,1
3,0,3,27390,26960,23840,...,3379.0,2024,10,1,1
4,0,4,24450,22240,19600,...,3379.0,2024,10,1,1


In [129]:
columns_will_convert = [
    'Harga_lag_1',
    'Harga_lag_7',
    'Harga_lag_30',
    'Google_trends',
    'MYRUSD',
    'SGDIDR',
    'THBUSD',
    'USDIDR',
    'Harga'
]
data_forecast[columns_will_convert] = data_forecast[columns_will_convert].apply(pd.to_numeric, errors='coerce')

In [130]:
data_forecast.head()

,Bahan_pangan,Daerah,Harga_lag_1,Harga_lag_7,Harga_lag_30,...,SGDIDR,Year,Month,Day,Weekday
0,0,0,28960,27700,26390,...,3379.0,2024,10,1,1
1,0,1,24230,24240,21250,...,3379.0,2024,10,1,1
2,0,2,27940,25910,25520,...,3379.0,2024,10,1,1
3,0,3,27390,26960,23840,...,3379.0,2024,10,1,1
4,0,4,24450,22240,19600,...,3379.0,2024,10,1,1


In [131]:
data_forecast.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40664 entries, 0 to 40663
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Bahan_pangan   40664 non-null  int64  
 1   Daerah         40664 non-null  int64  
 2   Harga_lag_1    40664 non-null  int64  
 3   Harga_lag_7    40664 non-null  int64  
 4   Harga_lag_30   40664 non-null  int64  
 5   Google_trends  40664 non-null  int64  
 6   Harga          40664 non-null  int64  
 7   MYRUSD         40664 non-null  float64
 8   THBUSD         40664 non-null  float64
 9   USDIDR         40664 non-null  float64
 10  SGDIDR         40664 non-null  float64
 11  Year           40664 non-null  int32  
 12  Month          40664 non-null  int32  
 13  Day            40664 non-null  int32  
 14  Weekday        40664 non-null  int32  
dtypes: float64(4), int32(4), int64(7)
memory usage: 4.0 MB


In [132]:
# Pisahkan fitur dan target
X = data_forecast.drop('Harga', axis=1)
y = data_forecast['Harga']

In [133]:
# Get the rows matching your filter criteria
filter_mask = (data_forecast['Day'] == date_start.day) & \
            (data_forecast['Month'] == date_start.month) & \
            (data_forecast['Year'] == date_start.year) & \
            (data_forecast['Harga_lag_1'] != 0) & \
            (data_forecast['Harga_lag_7'] != 0) & \
            (data_forecast['Harga_lag_30'] != 0) & \
            (data_forecast['Harga'] == 0)
    
# Get the subset for prediction
X_subset = data_forecast[filter_mask].drop('Harga', axis=1)

In [134]:
X_subset.shape

(440, 14)

In [135]:
X_subset.head()

,Bahan_pangan,Daerah,Harga_lag_1,Harga_lag_7,Harga_lag_30,...,SGDIDR,Year,Month,Day,Weekday
0,0,0,28960,27700,26390,...,3379.0,2024,10,1,1
1,0,1,24230,24240,21250,...,3379.0,2024,10,1,1
2,0,2,27940,25910,25520,...,3379.0,2024,10,1,1
3,0,3,27390,26960,23840,...,3379.0,2024,10,1,1
4,0,4,24450,22240,19600,...,3379.0,2024,10,1,1


In [136]:
X_subset.tail()

,Bahan_pangan,Daerah,Harga_lag_1,Harga_lag_7,Harga_lag_30,...,SGDIDR,Year,Month,Day,Weekday
437,12,29,9620,9760,9800,...,3379.0,2024,10,1,1
438,12,30,10690,10760,10840,...,3379.0,2024,10,1,1
439,12,31,9920,9780,9800,...,3379.0,2024,10,1,1
440,12,32,8960,8940,9020,...,3379.0,2024,10,1,1
441,12,33,10640,10580,10730,...,3379.0,2024,10,1,1


In [137]:
def pipeline_preprocessing(X):
    # Buat pipeline dengan preprocessing
    pipeline = Pipeline([
        ('quantile_transform', QuantileTransformer(n_quantiles=400, output_distribution='normal')),
        ('scaler', StandardScaler())  # Scaling fitur
    ])
    pipeline.fit(X)
    X_transformed = pipeline.transform(X)
    return X_transformed

In [138]:
# Load the saved model
best_xgb = xgboost.XGBRegressor()
best_xgb.load_model('forecast_xgboost_regressor_model.json')

In [139]:
for index, date in enumerate([date_start + dt.timedelta(days=x) for x in range((date_end - date_start).days + 1)]):
    day = date.day
    month = date.month
    year = date.year
        
    # Get the rows matching your filter criteria
    filter_mask = (data_forecast['Day'] == day) & \
                  (data_forecast['Month'] == month) & \
                  (data_forecast['Year'] == year) & \
                  (data_forecast['Harga_lag_1'] != 0) & \
                  (data_forecast['Harga_lag_7'] != 0) & \
                  (data_forecast['Harga_lag_30'] != 0) & \
                  (data_forecast['Harga'] == 0)
    
    # Get the subset for prediction
    X_subset = data_forecast[filter_mask].drop('Harga', axis=1)
    
        # Only predict if there are rows to predict
    if len(X_subset) > 0:
        X_subset_transformed = pipeline_preprocessing(X_subset)
        result = best_xgb.predict(X_subset_transformed)
        # Update using the same filter mask
        data_forecast.loc[filter_mask, 'Harga'] = result.round().astype(int)
        for (bahan_pangan, daerah), group in data_forecast.groupby(['Bahan_pangan', 'Daerah']):
            # Sort by date components
            group_sorted = group.sort_values(by=['Year', 'Month', 'Day'])
            
            # Get the indices in sorted order
            indices = group_sorted.index
            
            # Update lag_1
            for i in range(1, len(indices)):
                current_idx = indices[i]
                prev_idx = indices[i-1]
                
                # If Harga_lag_1 is 0, update it with the previous day's Harga
                if data_forecast.loc[current_idx, 'Harga_lag_1'] == 0:
                    data_forecast.loc[current_idx, 'Harga_lag_1'] = data_forecast.loc[prev_idx, 'Harga']
            
            # Update lag_7
            for i in range(7, len(indices)):
                current_idx = indices[i]
                prev_7_idx = indices[i-7]
                
                # If Harga_lag_7 is 0, update it with the value from 7 days ago
                if data_forecast.loc[current_idx, 'Harga_lag_7'] == 0:
                    data_forecast.loc[current_idx, 'Harga_lag_7'] = data_forecast.loc[prev_7_idx, 'Harga']
            
            # Update lag_30
            for i in range(30, len(indices)):
                current_idx = indices[i]
                prev_30_idx = indices[i-30]
                
                # If Harga_lag_30 is 0, update it with the value from 30 days ago
                if data_forecast.loc[current_idx, 'Harga_lag_30'] == 0:
                    data_forecast.loc[current_idx, 'Harga_lag_30'] = data_forecast.loc[prev_30_idx, 'Harga']
            # 
            # print(f"Remaining zeros in Harga_lag_1: {(data_forecast['Harga_lag_1'] == 0).sum()}")
            # print(f"Remaining zeros in Harga_lag_7: {(data_forecast['Harga_lag_7'] == 0).sum()}")
            # print(f"Remaining zeros in Harga_lag_30: {(data_forecast['Harga_lag_30'] == 0).sum()}")

In [140]:
data_forecast.head()

,Bahan_pangan,Daerah,Harga_lag_1,Harga_lag_7,Harga_lag_30,...,SGDIDR,Year,Month,Day,Weekday
0,0,0,28960,27700,26390,...,3379.0,2024,10,1,1
1,0,1,24230,24240,21250,...,3379.0,2024,10,1,1
2,0,2,27940,25910,25520,...,3379.0,2024,10,1,1
3,0,3,27390,26960,23840,...,3379.0,2024,10,1,1
4,0,4,24450,22240,19600,...,3379.0,2024,10,1,1


In [141]:
data_forecast.tail()

,Bahan_pangan,Daerah,Harga_lag_1,Harga_lag_7,Harga_lag_30,...,SGDIDR,Year,Month,Day,Weekday
40659,12,29,9950,9950,9950,...,3508.0,2024,12,31,1
40660,12,30,11130,11136,11136,...,3508.0,2024,12,31,1
40661,12,31,10050,10055,10050,...,3508.0,2024,12,31,1
40662,12,32,8678,8678,8678,...,3508.0,2024,12,31,1
40663,12,33,10908,10908,10908,...,3508.0,2024,12,31,1


In [142]:
data_forecast.to_csv('data_forecast_final.csv', index=False)

In [149]:
for (bahan_pangan, daerah), group in data_forecast.groupby(['Bahan_pangan', 'Daerah']):
    print(group.head(10))
    break

      Bahan_pangan  Daerah  Harga_lag_1  Harga_lag_7  Harga_lag_30  ...  SGDIDR  Year  Month  Day  Weekday
0                0       0        28960        27700         26390  ...  3379.0  2024     10    1        1
442              0       0        31222        28110         26730  ...  3380.0  2024     10    2        2
884              0       0        34730        28110         26240  ...  3380.0  2024     10    3        3
1326             0       0        35865        27890         26110  ...  3405.0  2024     10    4        4
1768             0       0        30151        28150         26080  ...  3423.0  2024     10    5        5
2210             0       0        30172        28430         26150  ...  3438.0  2024     10    6        6
2652             0       0        30580        28960         25950  ...  3446.0  2024     10    7        0
3094             0       0        30765        31222         25490  ...  3422.0  2024     10    8        1
3536             0       0        308